# File sorting

The cells below will move data-cube stats tile based results into folders labelled by seasonal quarter (i.e. one folder called `DJF`, another folder called `JFM` etc.)

It will then generate a `.vrt` and a mosaic `.tif` for each season

In [1]:
import fnmatch
import os
from multiprocessing import Pool

In [2]:
#create a list of seasons formated as "*_.DJF.tif" so 
#we can aggregate files by season
zz = ['JFM',
    'FMA',
    'MAM',
    'AMJ',
    'MJJ',
    'JJA',
    'JAS',
    'ASO',
    'SON',
    'OND',
    'NDJ',
    'DJF']

seasons = []
for z in zz:
    seasons.append('*_'+ z + '.tif')

In [3]:
directory='../results/NSW_NDVI_Climatologies_std/'
for file in os.listdir(directory):
    for s in seasons:
        #create folder for each season
        if not os.path.exists(directory + s[2:5]):
            os.mkdir(directory +s[2:5]) 
        #find tiles with matching seasons and move them to the correct folder    
        if fnmatch.fnmatch(file, s): 
            os.rename(directory +file, directory + s[2:5] + '/' + file)

## Build .vrt and mosaic tiff for each year
The following couple of cells will create `.vrt` files (which are used to read large numbers of geotiffs into QGIS), then, using the `.vrt` file, gdal is used to create a sigle `mosaicked geotiff` of all the tiles for each year. Beware: this can be very slow if you have compressed multi-band geotiffs.

In [4]:
cpus=4
directory = '/g/data1a/r78/cb3058/dea-notebooks/vegetation_anomalies/results/NSW_NDVI_Climatologies_std/'
prefix= "ndvi_clim_std"

In [5]:
#build vrt
for folder in os.listdir(directory):
    os.chdir(directory + folder)
    try:
        os.system("gdalbuildvrt " + prefix +"_"+folder + "_nsw_mosaic.vrt *.tif")
        print("building .vrt for " + folder)
    except:
        print("passing " +folder)
        continue


building .vrt for AMJ
building .vrt for NDJ
building .vrt for MAM
building .vrt for JJA
building .vrt for SON
building .vrt for OND
building .vrt for JFM
building .vrt for ASO
building .vrt for JAS
building .vrt for FMA
building .vrt for MJJ
building .vrt for DJF


In [6]:
#multithreaded version of mosaic
def mosaicTiffs(folder):
    if not folder.startswith('.'): #ignore hidden fodler
        os.chdir(directory + folder)
        print("building mosaic for " + folder)
        os.system("gdal_translate "\
           "-co BIGTIFF=YES "\
           "-co COMPRESS=DEFLATE "\
           "-co ZLEVEL=9 "\
           "-co PREDICTOR=1 "\
           "-co TILED=YES "\
           "-co BLOCKXSIZE=1024 "\
           "-co BLOCKYSIZE=1024 "\
           + prefix +"_"+folder + "_nsw_mosaic.vrt " + prefix + "_" + folder + "_nsw_mosaic.tif")


In [22]:
folder = os.listdir(directory)
folder.sort()

pool = Pool(cpus)  
pool.map(mosaicTiffs, x)

building mosaic for OND
building mosaic for AMJ


[None, None]